# Notebook 7a: no plot output

In [1]:
import os
from pathlib import Path
import pickle

In [2]:
Nb_name= 'Notebook_7_a'

In [3]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

HowManyBack = 1

seq_str= "0a1b2a3b4abcde5a6a7a"

data_dir_output = "/Users/gilmandelbaum/Desktop/outPut"


In [4]:
# Parameters
data_dir_input = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.dual_color_photo_full_data_set/"
data_dir_output = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis/"
drop2blocks = "yes"
drop2lastblocks = "yes"
seq_str = "0a1b2a3a4abcd5a6a7a"
HowManyBack = 1
Nb0 = "a"
Nb1 = "b"
Nb2 = "a"
Nb3 = "a"
Nb4 = ["a", "b", "c", "d"]
Nb5 = "a"
Nb6 = "a"
Nb7 = "a"
Tags = [
    "Reward_NoReward_tag",
    "ENL_tag",
    "Cue_to_Sel_tag",
    "Cue_to_Sel_fromPrevTrial_tag",
    "firstENLp_tag",
]
Timing = [600, 200, 15, 15, 100]
Start_or_end = ["start", "end", "start", "start", "start"]
y_axis_list = ["df_f", "z_score", "z_score_sliding_window"]
mouse = "T187"
data_day = "2019_03_28__T187"
date = "2019_03_28"
photo_day = "T187-152018"


In [5]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+Nb_name+'_seq'+seq_str+'.pickle'
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump((),my_file)